In [ ]:
import wandb
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
plt.style.use("paper")
plt.style.use("wiley")

In [ ]:
api = wandb.Api()

In [ ]:
runs = api.runs(
    "tobifinn/test_diffusion_nextsim_regional",
    filters={
        "tags": "freedrift"
    }
)

In [ ]:
results = pd.DataFrame()

for run in runs:
    table = [artifact for artifact in run.logged_artifacts() if artifact.type == 'run_table'][0]
    table = table.get("test/scores.table.json")
    table = pd.DataFrame(table.data, columns=table.columns)
    results[run.name] = table["rmse_sit"]

In [ ]:
det_run = api.run(
    "tobifinn/test_diffusion_nextsim_regional/ri3q51mi"
)
det_table = [artifact for artifact in det_run.logged_artifacts() if artifact.type == 'run_table'][0]
det_table = det_table.get("test/scores.table.json")
det_table = pd.DataFrame(det_table.data, columns=det_table.columns)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2), dpi=300)
ax.grid()
ax.plot(np.arange(31)/2, results["freedrift_nearest"], c="#DCA189", ls="dashed", label="Nearest")
ax.plot(np.arange(31)/2, results["freedrift_linear"], c="#FF7A45", ls="-.", label="Linear")
ax.plot(np.arange(31)/2, results["freedrift_cubic"], c="#AB7A67", ls="dotted", label="Cubic")
ax.plot(np.arange(31)/2, results["freedrift_perfect"], c="black", ls="-", label="Perfect linear")
ax.plot(np.arange(31)/2, det_table["rmse_sit"], ls="-", c="#81B3D5", alpha=1., label="Deterministic")
ax.legend(framealpha=1, fancybox=False, ncol=2, loc=8, bbox_to_anchor=(0.5, 0.85))
ax.set_xlim(0, 15)
ax.set_xlabel("Lead time (days)")
ax.set_ylim(0, 0.5)
ax.set_ylabel("RMSE SIT (m)")
fig.savefig("figures/fig_app_a3_freedrift_sit.png", dpi=300)